In [4]:
import doctest
import numpy as np
from itertools import permutations


def distance(point1, point2):
    """
    Returns the Euclidean distance of two points in the Cartesian Plane.

    >>> distance([3,4],[0,0])
    5.0
    >>> distance([3,6],[10,6])
    7.0
    """
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2) ** 0.5


def total_distance(points):
    """
    Returns the length of the path passing throught
    all the points in the given order.

    >>> total_distance([[1,2],[4,6]])
    5.0
    >>> total_distance([[3,6],[7,6],[12,6]])
    9.0
    """
    return sum([distance(point, points[index + 1]) for index, point in enumerate(points[:-1])])


def travelling_salesman(points, start=None):
    """
    Finds the shortest route to visit all the cities by bruteforce.
    Time complexity is O(N!), so never use on long lists.

    >>> travelling_salesman([[0,0],[10,0],[6,0]])
    ([0, 0], [6, 0], [10, 0])
    >>> travelling_salesman([[0,0],[6,0],[2,3],[3,7],[0.5,9],[3,5],[9,1]])
    ([0, 0], [6, 0], [9, 1], [2, 3], [3, 5], [3, 7], [0.5, 9])
    """
    if start is None:
        start = points[0]
    return min([perm for perm in permutations(points) if perm[0] == start], key=total_distance)

def haversine_enum(item1, item2):
    """
    Returns the great-circle distance between two enumearted points
    on a sphere given their indexes, longitudes, and latitudes in the
    form of a tuple.

    >>> haversine_enum((0, (3,5)), (1, (4,7)))
    154.27478490048566

    >>> haversine_enum((0, (41.325, -72.325)), (1, (41.327, -72.327)))
    0.17282397386672291
    """
    r = 3959 #radius of the earth
    r_lat = np.radians(item1[1][0])
    r_lat2 = np.radians(item2[1][0])
    delta_r_lat = np.radians(item2[1][0]-item1[1][0])
    delta_r_lon = np.radians(item2[1][1]-item1[1][1])

    a = (np.sin(delta_r_lat / 2) * np.sin(delta_r_lat / 2) +
        np.cos(r_lat) * np.cos(r_lat2) *
        np.sin(delta_r_lon / 2) * np.sin(delta_r_lon / 2))

    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    d = r * c

    return d


def optimized_travelling_salesman_enum(points, start=None):
    """
    Taken from:
        https://codereview.stackexchange.com/questions/81865/
        travelling-salesman-using-brute-force-and-heuristics

    As solving the problem in the brute force way is too slow,
    this function implements a simple heuristic: always
    go to the nearest city.

    Even if this algoritmh is extremely simple, it works pretty well
    giving a solution only about 25% longer than the optimal one (cit. Wikipedia),
    and runs very fast in O(N^2) time complexity.
    """

    points = list(enumerate(points))
    if start is None:
        start = points[0]

    must_visit = points
    path = [start]
    must_visit.remove(start)

    while must_visit:
        nearest = min(must_visit,
                       key=lambda x: haversine_enum(path[-1], x))
        path.append(nearest)
        must_visit.remove(nearest)

    return path


def main():
    doctest.testmod()
    points = [[0, 0], [1, 5.7], [2, 3], [3, 7],
              [0.5, 9], [3, 5], [9, 1], [10, 5]]
    print("""The minimum distance to visit all the following points: {}
starting at {} is {}.

The optimized algoritmh yields a path long {}.""".format(
        tuple(points),
        points[0],
        total_distance(travelling_salesman(points)),
        total_distance(optimized_travelling_salesman(points))))


if __name__ == "__main__":
    main()

**********************************************************************
File "__main__", line 54, in __main__.haversine_enum
Failed example:
    haversine_enum((0, (41.325, -72.325)), (1, (41.327, -72.327)))
Expected:
    0.17282397386672291
Got:
    0.1728239738667229
**********************************************************************
1 items had failures:
   1 of   2 in __main__.haversine_enum
***Test Failed*** 1 failures.
The minimum distance to visit all the following points: ([0, 0], [1, 5.7], [2, 3], [3, 7], [0.5, 9], [3, 5], [9, 1], [10, 5])
starting at [0, 0] is 25.90302275027582.

The optimized algoritmh yields a path long 27.995524884656632.
